# библиотеки и константы

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np
import random
import datetime
from datetime import datetime
import warnings
import os, fnmatch
import yadisk
import shutil
import psycopg2
from sqlalchemy import create_engine
from matplotlib import pyplot as plt 
import seaborn as sns
from tqdm import tqdm

warnings.simplefilter('ignore')
options = webdriver.ChromeOptions()
now = datetime.now()
options.headless = True

token = '*****'
dir_ = "*****"

EXE_PATH = r'C:\chromedriver.exe'

dates = {
    'января':'01',
    'февраля':'02',
    'марта':'03',
    'апреля':'04',
    'мая':'05',
    'июня':'06',
    'июля':'07',
    'августа':'08',
    'сентября':'09',
    'октября':'10',
    'ноября':'11',
    'декабря':'12'
}

host_2 = '*****'
user_2 = '*****'
password_2 = '*****'
db_name_2 = '*****'
port_2 = 5432

connection_2 = psycopg2.connect(
host=host_2,
    user=user_2,
    password=password_2,
    database = db_name_2,
    port=port_2
)


connection_2.autocommit = True

SQL_insert = "INSERT INTO apartment_ishevsk (adres, area, rooms, square, floor, floors_number,\
build_year, house_type, date, price, m_price, description, url_link, ad_number)\
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

db_config = {'user': '*****', # имя пользователя
'pwd': '*****', # пароль
'host': '*****',
'port': 5432, # порт подключения
'db': '*****'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

engine = create_engine(connection_string)

y = yadisk.YaDisk(token=token)
print(y.check_token())

True


# Функции

In [13]:
def get_date_type(date):
    try:
        return datetime.strptime(date, "%d.%m.%Y")
    except:        
        return datetime.strptime(date[:10], "%Y-%m-%d")

def yandex():
    print('Начинаем перенос скринов на Я.диск')
    # Собираем файлы
    files = []
    listOfFiles = os.listdir('screenshots')  
    pattern = "*.png"  
    for entry in listOfFiles:  
        if fnmatch.fnmatch(entry, pattern):            
                files.append(entry)
    # переносим в другую директорию
    for file in files:
        shutil.move('screenshots' +'\\' + file, os.getcwd())
    #записываем на Я.Диск
    c = 0        
    for file in files:
        print(f'Записываем {file}')
        if y.is_file(f'{dir_}{file}'):
            print('Такой уже есть')
            continue
        else:
            c+=1
            y.upload(file, f'{dir_}{file}')
    #удаляем файлы       
    files = []
    listOfFiles = os.listdir('.')  
    pattern = "*.png"  
    for entry in listOfFiles:  
        if fnmatch.fnmatch(entry, pattern):            
                files.append(entry)
    for file in files:
        os.remove(file)

    print(f'{c} файлов записано на Я.Диск')

def get_pages_number():
    pages = driver.find_elements("class name", "pagination-item-JJq_j")
    pages_ = []
    for page in pages:    
        if page.text.isdigit():        
            pages_.append(int(page.text))
    return(max(pages_))

def get_norm_links():
    all_links = []
    elems = driver.find_elements("xpath" ,"//a[@href]")
    for elem in elems:
        all_links.append(elem.get_attribute("href"))
    norm_links = []
    for link in all_links:
        if link.split('_')[-1].isdigit():
            norm_links.append(link)
    return(list(set(norm_links)))


def is_bad_adv(i,j):
    try:
        try:
            driver.find_element("class name", "closed-warning-content-_f4_B").text
            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
            print(url)
            print('Снято с публикации')
            return(True)
        except:
            pass
        try:
            driver.find_element("class name", "desktop-16c9mq7").text
            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
            print(url)
            print('Страницы не существует')
            return(True)
        except:
            pass

        try:
            driver.find_element("class name", "closed-warning-content-2ooy4").text
            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
            print('Снято с публикации')
            print(url)
            return(True)
        except:
            pass

        try:
            driver.find_element("class name", "page-title-count-wQ7pG").text
            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
            print(url)
            print('Страница поиска, объявления нет')
            return(True)
        except:
            pass
        try:
            driver.find_element("class name", "b-404").text
            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
            print('Нет такой страницы')
            print(url)
            return(True)
        except:
            return(False)
    except:
        return False

def get_rayon():
    try:
       
        try:
            rayon = driver.find_element("class name", "style-item-address-georeferences-item-18pFt")
            return(rayon.text)
        except:
            pass
        
        try:
            rayon = driver.find_element("class name", "style-item-address-georeferences-item-TZsrp")
            return(rayon.text)
        except:
            pass
    
    except:        
        return(None)
        
def get_adress():
    try:
        try:
            adress = driver.find_element("class name", "style-item-address__string-3Ct0s")
            return(adress.text)
        except:
            pass
        try:
            adress = driver.find_element("class name", "style-item-address__string-wt61A")
            return(adress.text)
        except:
            pass
    except:        
        return(None)
        
def get_params():
    try:
        if len(driver.find_elements("class name", "params-paramsList__item-1Xeok")) > 0:
            params = driver.find_elements("class name", "params-paramsList__item-1Xeok")
            params_ = []
            for param in params:
                params_.append(param.text.split(':'))
            param_list = []
            for param in params_:
                  param_list.extend(param)
            return param_list
        
        if len(driver.find_elements("class name", "params-paramsList-zLpAu")) > 0:
            param_list = driver.find_elements("class name", "params-paramsList-zLpAu")[0].text.replace('\n', ' ').split(' ')
            return param_list
    except:
        print('Ошибка в параметрах')
        print(url)
        return('ERRROR')
    
def get_house_params():
    try:
        if len(driver.find_elements("class name", "style-item-params-list-3YJu7")) > 0:
            house_params = driver.find_elements("class name", "style-item-params-list-3YJu7")
            house_params_ = []

            for param in house_params:
                house_params_.append(param.text.replace('\n',' '))
            if len(house_params_) > 0:
                return(house_params_[0].split(' '))
            else:
                return('ERROR')
        if len(driver.find_elements("class name", "style-item-params-McqZq")) > 0:
            return driver.find_elements("class name", "style-item-params-McqZq")[0].text.replace('\n', ' ').split(' ')[2:]
    except:
        print('Ошибка в параметрах')
        print(url)
        return('ERRROR')

def get_room(kv_par):
    try:
        try:
            x = kv_par[kv_par.index('комнат:')+1]
            return(x)
        except:
            pass
        try:
            x = int(kv_par[kv_par.index('Количество комнат')+1].strip())
            return(x)
        except:
            pass
        try:
            x = kv_par[kv_par.index('Количество комнат')+1].strip()
            return(x)
        except:
            pass
    except:        
        return(None)

def get_house_type(house_par): 
    try:
        try:
            x = house_par[house_par.index('дома:')+1]
            return(x)
        except:
            print('Ошибка в типе дома')
            print(url)
            return(None)
    except:       
        return(None)

def get_floors(house_par):
    try:
        try:
            x = int(house_par[house_par.index('доме:')+1].strip())
            return(x)
        except:            
            return(None)
    except:        
        return(None)

def get_kv_floor(kv_par):
    try:
        try:
            x = int(kv_par[kv_par.index('Этаж:')+1].strip())
            return(x)
        except:
            pass
        try:
            x = kv_par[kv_par.index('Этаж')+1].strip()
            return(int(x.split()[0].strip()))
        except:
            pass
    except:       
        return(None)

def get_square(kv_par):
    try:
        try:
            x = float(kv_par[kv_par.index('Общая площадь')+1].split()[0])
            return(x)
        except:
            pass
        try:
            x = float(kv_par[kv_par.index('площадь:')+1])
            return(x)
        except:
            pass
    except:        
        return(None)

def get_year(house_par):
    try:
        x = int(house_par[house_par.index('постройки:')+1])
        return(x)
    except:
        return(None)

def get_price():
    try:
        try:
            test = driver.find_element("class name", "style-item-view-price-content-1aHxn")
            x = test.text.replace('\n', '').replace(' ', '').split('₽')[0]
            return float(x)
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-price-PuQ0I")
            x = test.text.replace('\n', '').replace(' ', '').split('₽')[0]
            return float(x)
        except:
            pass
    except:
        return None
    
def get_view():
    try:
        try:
            test = driver.find_element("class name", "style-item-description-html-qCwUL")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-text-mc3G6")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-html-1_RNo")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-text-SzN56")
            return(test.text.replace('\n', ' '))
        except:
            pass
    except:        
        return(None)

def get_date():
    try:
        #1 ексепт
        try:
            test = driver.find_element("class name", "style-item-metadata-date-1y5w6")
            return(' '.join(test.text.split()[:2]) + ' ' + str(time.localtime().tm_year))
        except:
            pass
        
        # 2 ексепт
        try:
            test = driver.find_element("class name", "style-item-metadata-date-EEthc")
            return(' '.join(test.text.split()[:2]) + ' ' + str(time.localtime().tm_year))
        except:
            pass
        
        #3 ексепт
        try:
            test = driver.find_element("class name", "style-item-footer-3uXQz")
            if 'сегодн' in test.text.split()[3]:
                return(str(now.day) +'.'+  str(now.month)+'.'+ str(now.year))
            if 'вчер' in test.text.split()[3]:                
                return(str(now.day-1) +'.'+  str(now.month)+'.'+ str(now.year))
            else:
                date = (test.text.split()[3]+ ' '+ test.text.split()[4]+' '+ str(now.year))
                date = date.split()
                date[1] = dates[date[1]]
                return('.'.join(date))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-footer-Ufxh_")
            if 'сегодн' in test.text.split()[3]:
                return(str(now.day) +'.'+  str(now.month)+'.'+ str(now.year))
            if 'вчер' in test.text.split()[3]:                
                return(str(now.day-1) +'.'+  str(now.month)+'.'+ str(now.year))
            else:
                date = (test.text.split()[3]+ ' '+ test.text.split()[4]+' '+ str(now.year))
                date = date.split()
                date[1] = dates[date[1]]
                return('.'.join(date))
        except:
            pass
            
    except:        
        return(None)

# Links

In [4]:
driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
driver.get('https://www.avito.ru/izhevsk/kvartiry/prodam/vtorichka-ASgBAQICAUSSA8YQAUDmBxSMUg?p=1')
page_numbers = get_pages_number()
print(f'Всего страниц: {page_numbers}')

links = []

for i in tqdm(range(1, page_numbers+1)):
    driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
    driver.get(f'https://www.avito.ru/izhevsk/kvartiry/prodam/vtorichka-ASgBAQICAUSSA8YQAUDmBxSMUg?p={i}')
    links.extend(get_norm_links())
    driver.close()
    time.sleep(random.randint(5,12))

links = set(links)

with connection_2.cursor() as cursor:
                    cursor.execute(
                    'TRUNCATE TABLE apartment_new_links'
                    )

query = "SELECT * FROM apartment_ishevsk"
old_links = list(pd.io.sql.read_sql(query, con = engine)['url_link'].values)

for link in links:
    if link in old_links:
        continue
    else:
        with connection_2.cursor() as cursor:
                    cursor.execute(
                    f'''INSERT INTO apartment_new_links (links) VALUES ('{link}')'''
                    )
query = "SELECT links FROM apartment_new_links"
print(f'Новых линков: {len(list(pd.io.sql.read_sql(query, con = engine)["links"].values))}')

  0%|                                                                                           | 0/82 [00:00<?, ?it/s]

Всего страниц: 82


100%|████████████████████████████████████████████████████████████████████████████████| 82/82 [1:07:46<00:00, 49.59s/it]


Новых линков: 373


# Парсинг карточек

In [16]:
# сброс счетчика

counts_ = '0'

with connection_2.cursor() as cursor:
    cursor.execute(
    f'''UPDATE apartment_counts
    SET counts = {counts_}'''
    )

In [17]:
query = "SELECT * FROM apartment_counts"
test = pd.io.sql.read_sql(query, con = engine) 
i = test['counts'].values[0]
i

0

## Сам парсер

In [19]:

query = "SELECT * FROM apartment_new_links"
test = pd.io.sql.read_sql(query, con = engine) 
links = list(test['links'].values)

query = "SELECT * FROM apartment_counts"
test = pd.io.sql.read_sql(query, con = engine) 
i = test['counts'].values[0]
# Пустой ДФ

df = {}
df['Адрес'] = []
df['Район'] = []
df['Количество комнат'] = []
df['Общая площадь'] = []
df['Этаж'] = []
df['Этажность'] = []
df['Год постройки'] = []
df['Тип дома'] = []
df['Дата объявления'] = []
df['Стоимость'] = []
df['Метр квадратный'] = []
df['Описание'] = []
df['Ссылка'] = []
df['номер объявления'] = []

options.headless = True
c = 0
for j,url in enumerate (links[i:]):
    if 'auktsion' in url:
        print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
        print('Аукционы в жопу')
        continue
    driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
    driver.get(url)
    
    
    if j % 5 == 0 and j > 1:
              
        counts_ = str(i+j)

        with connection_2.cursor() as cursor:
            cursor.execute(
            f'''UPDATE apartment_counts
            SET counts = {counts_}'''
            )
        
        
        print(f'Данные и счетчик записаны на {j+i} позиции')
        df = pd.DataFrame(df)
        for q in range (len(df)):
            values_ = []
            for value in df.loc[q].values:
                values_.append(str(value))
            with connection_2.cursor() as cursor:
                    cursor.execute(
                    SQL_insert, [*values_]
                    )
        print(f'Записано в бд')
        yandex()
        df = {}
        df['Адрес'] = []
        df['Район'] = []
        df['Количество комнат'] = []
        df['Общая площадь'] = []
        df['Этаж'] = []
        df['Этажность'] = []
        df['Год постройки'] = []
        df['Тип дома'] = []
        df['Дата объявления'] = []
        df['Стоимость'] = []
        df['Метр квадратный'] = []
        df['Описание'] = []
        df['Ссылка'] = []
        df['номер объявления'] = []
        # записываем счетчик
        
    if is_bad_adv(i,j): 
        driver.close()
        time.sleep(2)
        continue     
    try:            
        kv_par = get_params()
        house_par = get_house_params()
        #print('---'*22)
        df['Адрес'].append(get_adress())
        df['Район'].append(get_rayon())
        df['Количество комнат'].append(get_room(kv_par))
        df['Общая площадь'].append(get_square(kv_par))
        df['Этаж'].append(get_kv_floor(kv_par))
        df['Этажность'].append(get_floors(house_par))
        df['Год постройки'].append(get_year(house_par))
        df['Тип дома'].append(get_house_type(house_par))
        df['Стоимость'].append(get_price())
        try:
            df['Метр квадратный'].append(df['Стоимость'][-1]/df['Общая площадь'][-1])
        except:
            print('Ошибка в определении стоимости метра кв.')
            df['Метр квадратный'].append(None)
        df['Дата объявления'].append(get_date())
        df['Ссылка'].append(url)
        df['Описание'].append(get_view())
        df['номер объявления'].append(df['Ссылка'][-1].split('_')[-1])
        
        print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
        if df["Адрес"][-1] == None:
            print(f'Адрес: {df["Адрес"][-1]}')
        if df["Район"][-1] == None:
            print(f'Район: {df["Район"][-1]}')
        if df["Количество комнат"][-1] == None:
            print(f'Количество комнат: {df["Количество комнат"][-1]}')
        if df["Общая площадь"][-1] == None:
            print(f'Общая площадь: {df["Общая площадь"][-1]}')
        if df["Этаж"][-1] == None:
            print(f'Этаж: {df["Этаж"][-1]}')
        if df["Этажность"][-1] == None:
            print(f'Этажность: {df["Этажность"][-1]}')
        if df["Год постройки"][-1] == None:
            print(f'Год постройки: {df["Год постройки"][-1]}')
        if df["Тип дома"][-1] == None:
            print(f'Тип дома: {df["Тип дома"][-1]}')
        if df["Стоимость"][-1] == None:
            с+=1
            print(f'Стоимость: {df["Стоимость"][-1]}')
        else:
            c = 0
        if df["Дата объявления"][-1] == None:
            print(f'Дата объявления: {df["Дата объявления"][-1]}')
        if df["Описание"][-1] == None:
            print(f'Описание: {df["Описание"][-1]}')
        print(f'Ссылка: {df["Ссылка"][-1]}')
                        
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element('tag name' ,'body').screenshot(f'screenshots/{url.split("_")[-1]}.png')
        print('Скриншот сделан')
        driver.quit()
        
        print()
        
        #print('---'*22)
        

    except:
        print('Тут что-то пошло не так (ошибка с вылетом скрипта):')
        print(url)         
        break
        
        #j +=1
    
    time.sleep(random.randint(10,19))
    

else:
    yandex()
    df = pd.DataFrame(df)
    for q in range (len(df)):
        values_ = []
        for value in df.loc[q].values:
            values_.append(str(value))
        with connection_2.cursor() as cursor:
                cursor.execute(
                SQL_insert, [*values_]
                )
    print('###'*22)
    print('#'+' '*64+'#')
    print('#'+ ' ' *20 + 'СКРИПТ ОКОНЧИЛ РАБОТУ!' + ' '*22 + '#')
    print('#'+' '*64+'#')
    print('###'*22)

****************************************** № 290 из 373 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kvartiry/2-k._kvartira_648m_710et._2560991544
Скриншот сделан

****************************************** № 291 из 373 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kvartiry/1-k._kvartira_31m_39et._2602493388
Скриншот сделан

****************************************** № 292 из 373 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kvartiry/2-k._kvartira_629m_817et._2564760983
Скриншот сделан

****************************************** № 293 из 373 ******************************************
Год постройки: None
Ссылка: https://www.avito.ru/izhevsk/kvartiry/1-k._kvartira_328m_99et._2526240792
Скриншот сделан

****************************************** № 294 из 373 ******************************************
Год постройки: None
Ссылка: https://www.avito.ru/izhevsk/kvartiry/1-k._kvartira_364m_224

Скриншот сделан

****************************************** № 322 из 373 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kvartiry/3-k._kvartira_673m_410et._2534143819
Скриншот сделан

****************************************** № 323 из 373 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kvartiry/1-k._kvartira_333m_59et._2556696364
Скриншот сделан

****************************************** № 324 из 373 ******************************************
Год постройки: None
Ссылка: https://www.avito.ru/izhevsk/kvartiry/2-k._kvartira_45m_35et._2526455980
Скриншот сделан

Данные и счетчик записаны на 325 позиции
Записано в бд
Начинаем перенос скринов на Я.диск
Записываем 2526455980.png
Записываем 2534143819.png
Записываем 2544497694.png
Записываем 2556696364.png
Записываем 2561040452.png
5 файлов записано на Я.Диск
****************************************** № 325 из 373 ******************************************
Год постройки: None
Сс

Скриншот сделан

Данные и счетчик записаны на 355 позиции
Записано в бд
Начинаем перенос скринов на Я.диск
Записываем 2427629206.png
Записываем 2465685979.png
Записываем 2501690897.png
Записываем 2533210925.png
Записываем 2560957888.png
5 файлов записано на Я.Диск
****************************************** № 355 из 373 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kvartiry/2-k._kvartira_436m_34et._2587306464
Скриншот сделан

****************************************** № 356 из 373 ******************************************
Год постройки: None
Ссылка: https://www.avito.ru/izhevsk/kvartiry/3-k._kvartira_569m_15et._2533220533
Скриншот сделан

****************************************** № 357 из 373 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kvartiry/2-k._kvartira_62m_718et._2568812187
Скриншот сделан

****************************************** № 358 из 373 ******************************************
Район: None
Год постро

# Предобработка

In [20]:
query = "SELECT * FROM apartment_ishevsk"
df = pd.io.sql.read_sql(query, con = engine)
df = df[df.columns[1:]]
df.head()

,adres,area,rooms,square,floor,floors_number,build_year,house_type,date,price,m_price,description,url_link,ad_number
0,"ул. им. Репина, д. 35",р-н Индустриальный,3,75.1,2,16,nan,монолитный,15.08.2022,5602000.0,74593.87483355527,А где ты встречаешь свое утро? Представляем но...,https://www.avito.ru/izhevsk/kvartiry/3-k._kva...,2346094498
1,"Удмуртская Республика, Ижевск, Песочная улица, 32",р-н Октябрьский,3,63.0,5,5,1976.0,панельный,28.07.2022,2870000.0,45555.55555555556,Продам просторную трехкомнатную квартиру. Квар...,https://www.avito.ru/izhevsk/kvartiry/3-k._kva...,2258823609
2,"ул. Охотная, жилой дом",р-н Ленинский,2,60.0,5,17,nan,монолитный,23.8.2022,4875500.0,81258.33333333333,Застройщик NONAMEGROUP / Серия - Дома героев ...,https://www.avito.ru/izhevsk/kvartiry/2-k._kva...,2492497661
3,"ул. Постольская, стр. 2",р-н Ленинский,студия,54.0,13,18,nan,монолитный,23.8.2022,3830000.0,70925.92592592593,"Дом «Лето Красное» — это новый, строящийся жил...",https://www.avito.ru/izhevsk/kvartiry/kvartira...,2493265486
4,"ул. Холмогорова, уч. 113",р-н Октябрьский,студия,47.6,14,17,nan,монолитный,24.8.2022,4340000.0,91176.47058823529,ЖК «HAPPY»– это городская жизнь в окружении пр...,https://www.avito.ru/izhevsk/kvartiry/kvartira...,2470469778


In [21]:
df.duplicated(subset=['ad_number']).sum()

149

In [22]:
df = df.drop_duplicates()
cols = df.columns
for col in cols:
    print(f'{col}: {df[col].isna().sum()}')

adres: 0
area: 0
rooms: 0
square: 0
floor: 0
floors_number: 0
build_year: 0
house_type: 0
date: 0
price: 0
m_price: 0
description: 0
url_link: 0
ad_number: 0


In [23]:
for col in cols:
    print(f'{col}: {(df[col] == "nan").sum()}')

adres: 0
area: 129
rooms: 0
square: 2
floor: 2
floors_number: 2
build_year: 3436
house_type: 0
date: 0
price: 10
m_price: 12
description: 0
url_link: 0
ad_number: 0


In [24]:
df = df[df['square'] != 'nan']

In [25]:
for col in cols:
    print(f'{col}: {(df[col] == "None").sum()}')

adres: 0
area: 241
rooms: 0
square: 0
floor: 0
floors_number: 0
build_year: 18
house_type: 0
date: 0
price: 0
m_price: 0
description: 0
url_link: 0
ad_number: 0


In [26]:
df['square'] = df['square'].apply(float)
df['price'] = df['price'].apply(float)
df['m_price'] = df['m_price'].apply(float)
df['datetime'] = df['date'].apply(get_date_type)

In [27]:
df.to_excel('apartments_izhevsk.xlsx', index=0)